In [2]:
import dlib

In [3]:
!pip install face_recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100.1 MB 23 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=18797ed018aa565479c924184a6fbed7226b51ccb1581737c757275abc2e0aab
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [4]:
import face_recognition

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

In [24]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [25]:
trainDir = "/content/gdrive/MyDrive/Attendance_project/train"
testDir = "/content/gdrive/MyDrive/Attendance_project/test"

In [26]:
myList = os.listdir(trainDir)
print(myList)

studentNames = []
images = []
for student in myList:
  img = cv2.imread(f'{trainDir}/{student}')
  images.append(img)
  studentNames.append(os.path.splitext(student)[0])

print(studentNames)

['daniel.jpg', 'emma.jpg']
['daniel', 'emma']


In [27]:
def findEncodings(images):
  encodings = []
  for img in images:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    encoding = face_recognition.face_encodings(img)[0]
    encodings.append(encoding)
  return encodings

In [28]:
knownEncodings = findEncodings(images)
print(len(knownEncodings))

2


In [36]:
from google.colab.patches import cv2_imshow
from datetime import datetime

In [39]:
def markAttendance(name):
  with open('/content/gdrive/MyDrive/Attendance_project/attendance.csv','r+') as f:
    myDataList = f.readlines()
    nameList = []
    for line in myDataList:
      entry = line.split(',')
      nameList.append(entry[0])
      if name not in nameList:
        now = datetime.now()
        dtString = now.strftime('%H:%M:%S')
        f.writelines(f'n{name},{dtString}')

In [ ]:
cam = cv2.VideoCapture(0)

while True:
  success, image = cam.read()
  imgSmall = cv2.cvtColor(cv2.resize(img, (0, 0), None, 0.2, 0.2), cv2.COLOR_BGR2RGB)
  
  faceCurr = face_recognition.face_locations(imgSmall)
  encodeCurr = face_recognition.face_encodings(imgSmall, faceCurr)

  for face, loc in zip(encodeCurr, faceCurr):
    matches = face_recognition.compare_faces(knownEncodings, face)
    faceDis = face_recognition.face_distance(knownEncodings, face)
    print(faceDis)
    matchIndex = np.argmin(faceDis)

    if faceDis[matchIndex]< 0.50:
      name = studentNames[matchIndex].upper()
      markAttendance(name)
    else: name = 'Unknown'
    
    y1,x2,y2,x1 = loc
    y1, x2, y2, x1 = y1*4,x2*4,y2*4,x1*4
    cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
    cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
    cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)

  cv2_imshow()
  cv2.waitKey(1)